### Basic Stacking Model

**데이터 로딩**

In [1]:
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer_data = load_breast_cancer()

X_data = cancer_data.data
y_label = cancer_data.target

X_train, X_test, y_train, y_test = train_test_split(X_data, y_label, test_size = 0.2, random_state = 0)

**개별 Classifier와 최종 Stacking Data를 학습할 메타 Classifier 생성**

In [2]:
knn_clf = KNeighborsClassifier(n_neighbors = 4)
rf_clf = RandomForestClassifier(n_estimators = 100, random_state = 0)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators = 100)

# Stacking model을 위한 Classifier
lr_final = LogisticRegression(C = 10)

**개별 Classifier 학습/예측/평가**

In [3]:
knn_clf.fit(X_train, y_train)
rf_clf.fit(X_train, y_train)
dt_clf.fit(X_train, y_train)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=100)

In [4]:
# 학습된 개별 모델들의 예측 데이터 셋을 생성하고, 정확도 측정
knn_pred = knn_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)
dt_pred = dt_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)

print('KNN 정확도: {0:.4f}'.format(accuracy_score(y_test, knn_pred)))
print('랜덤 포레스트 정확도: {0:.4f}'.format(accuracy_score(y_test, rf_pred)))
print('결정 트리 정확도: {0:.4f}'.format(accuracy_score(y_test, dt_pred)))
print('에이다부스트 정확도: {0:.4f}'.format(accuracy_score(y_test, ada_pred)))

KNN 정확도: 0.9211
랜덤 포레스트 정확도: 0.9649
결정 트리 정확도: 0.8947
에이다부스트 정확도: 0.9561


**개별 model의 예측 결과를 meta model이 학습할 수 있도록 stacking 형태로 재 생성**

In [5]:
pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred])
print(pred.shape)

# transpose를 이용해 행과 열의 위치 교환, column level로 각 알고리즘의 예측 결과를 feature로 생성
pred = np.transpose(pred)
print(pred.shape)

(4, 114)
(114, 4)


**meta model 학습/예측/평가**

In [6]:
# test data로 학습하고, test data로 평가하므로 overfitting
lr_final.fit(pred, y_test)
final = lr_final.predict(pred)

print('최종 meta model의 예측 정확도: {0:.4f}'.format(accuracy_score(y_test, final)))

최종 meta model의 예측 정확도: 0.9737
